# SFC on Pynq

This notebook covers how to use low quantized Neural Networks on Pynq for inference on MNIST dataset by using SFC network composed of 4 fully connected layers with 256 neurons each.

- SFCW1A1 using 1 bit weights and 1 activation,

In [1]:
import os
print(os.environ["BOARD"])

Pynq-Zybo


In [2]:
import bnn

## 1. LFC and MNIST

This notebook performs inference on MNIST test set from http://yann.lecun.com/exdb/mnist/ which contains 10000 pictures of handwritten digits. The SFC network requires MNIST formatted input data, that's why the binary test file can be directly loaded. All other images have to be formatted to this specification (refer to BNN-PYNQ url and LFC webcam examples).

At first you need to download mnist test set and labels using wget and unzip the archive as shown below:
In order to be able to compare the inferred classes against the expected labels we first read the labels:

In [3]:
#get
!wget http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz 
!wget http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz 
#unzip
!gzip -d t10k-images-idx3-ubyte.gz
!gzip -d t10k-labels-idx1-ubyte.gz

--2020-04-25 20:45:14--  http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
Resolving yann.lecun.com (yann.lecun.com)... 104.28.6.204, 104.28.7.204, 2606:4700:3031::681c:6cc, ...
Connecting to yann.lecun.com (yann.lecun.com)|104.28.6.204|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1648877 (1.6M) [application/x-gzip]
Saving to: ‘t10k-images-idx3-ubyte.gz’

t10k-images-idx3-ub 100%[===================>]   1.57M  2.41MB/s    in 0.7s    

2020-04-25 20:45:15 (2.41 MB/s) - ‘t10k-images-idx3-ubyte.gz’ saved [1648877/1648877]

--2020-04-25 20:45:15--  http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz
Resolving yann.lecun.com (yann.lecun.com)... 104.28.7.204, 104.28.6.204, 2606:4700:3033::681c:7cc, ...
Connecting to yann.lecun.com (yann.lecun.com)|104.28.7.204|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4542 (4.4K) [application/x-gzip]
Saving to: ‘t10k-labels-idx1-ubyte.gz’

t10k-labels-idx1-ub 100%[===================>

In [3]:
#read labels
labels = []
print("Listing Labels . . .")
with open("/home/xilinx/jupyter_notebooks/bnn/t10k-labels-idx1-ubyte","rb") as lbl_file:
    #read magic number and number of labels (MSB first) -> MNIST header
    magicNum = int.from_bytes(lbl_file.read(4), byteorder="big")
    countLbl = int.from_bytes(lbl_file.read(4), byteorder="big")
    #now the labels are following byte-wise
    for idx in range(countLbl):
        labels.append(int.from_bytes(lbl_file.read(1), byteorder="big"))
    lbl_file.close()
len(labels)
print("Done!")

Listing Labels . . .
Done!


## 2. Hardware Inference

First of all a classifier needs to be instantiated. Using the LfcClassifier will allow to classify MNIST formatted images utilizing SFC network. There are two different runtimes available: hardware accelerated and pure software environment.

Once a classifier is instantiated the inference on MNIST images can be started using `classify_mnist` or `classify_mnists` methods - for both single and multiple images.

#### W1A1 - 1 bit weights and 1 bit activation

In [4]:
sfcW1A1_classifier = bnn.LfcClassifier(bnn.NETWORK_SFCW1A1,"mnist",bnn.RUNTIME_HW)
#lfcW1A1_classifier.classes

[DEBUG] starting FC classifier
[DEBUG] Datasets: ['chars_merged', 'streetview', 'cifar10', 'road-signs', 'mnist']
[DEBUG] ret: ['mnist']
[DEBUG] Initializing PynqBNN class . . . 
	runtime: python_hw
	network: sfcW1A1
[DEBUG] Overlay Loaded
[DEBUG] Loading python_hw-sfcW1A1-Zybo-Z7.so . . . 
[DEBUG] PynqBNN class initialized.
[DEBUG] Sourcing params at: /usr/local/lib/python3.6/dist-packages/bnn/params/mnist/sfcW1A1


In [5]:
result_W1A1 = sfcW1A1_classifier.classify_mnists("/home/xilinx/jupyter_notebooks/bnn/t10k-images-idx3-ubyte")

Inference took 25613.00 microseconds, 2.56 usec per image
Classification rate: 390426.73 images per second


## 3. Summary
### Accuracy

In [6]:
#compare against labels
countRight = 0
for idx in range(len(labels)):
    if labels[idx] == result_W1A1[idx]:
        countRight += 1
accuracyW1A1 = countRight*100/len(labels)

print("Accuracy SFCW1A1: ", accuracyW1A1)

Accuracy SFCW1A1:  95.7


## 4. Reset the device

In [7]:
from pynq import Xlnk

xlnk = Xlnk()
xlnk.xlnk_reset()